# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jul 18 2022 9:14AM,242046,10,0085877925,ISDIN Corporation,Released
1,Jul 18 2022 9:14AM,242046,10,0085877931,ISDIN Corporation,Released
2,Jul 18 2022 9:14AM,242046,10,0085877933,ISDIN Corporation,Released
3,Jul 18 2022 9:14AM,242046,10,0085877934,ISDIN Corporation,Released
4,Jul 18 2022 9:14AM,242046,10,0085877936,ISDIN Corporation,Released
5,Jul 18 2022 9:14AM,242046,10,0085877932,ISDIN Corporation,Released
6,Jul 18 2022 9:14AM,242046,10,0085877939,ISDIN Corporation,Released
7,Jul 18 2022 9:14AM,242046,10,0085877957,ISDIN Corporation,Released
8,Jul 18 2022 9:14AM,242046,10,0085877966,ISDIN Corporation,Released
9,Jul 18 2022 9:14AM,242046,10,0085877967,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
19,242046,Released,59
20,242047,Released,2
21,242048,Released,45
22,242049,Released,1
23,242050,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
242046,NaN,59.0
242047,NaN,2.0
242048,NaN,45.0
242049,NaN,1.0
242050,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
241994,16.0,31.0
242005,0.0,3.0
242007,0.0,1.0
242009,0.0,1.0
242012,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
241994,16,31
242005,0,3
242007,0,1
242009,0,1
242012,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,241994,16,31
1,242005,0,3
2,242007,0,1
3,242009,0,1
4,242012,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,241994,16,31
1,242005,,3
2,242007,,1
3,242009,,1
4,242012,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released      int32
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jul 18 2022 9:14AM,242046,10,ISDIN Corporation
59,Jul 18 2022 9:11AM,242050,20,Alumier Labs Inc.
60,Jul 18 2022 9:10AM,242048,12,Hush Hush
105,Jul 18 2022 9:09AM,242049,19,VITABIOTICS USA INC.
106,Jul 18 2022 9:09AM,242047,12,Hush Hush
108,Jul 18 2022 9:05AM,242045,10,ISDIN Corporation
161,Jul 18 2022 9:05AM,242043,10,ISDIN Corporation
222,Jul 18 2022 9:05AM,242041,10,ISDIN Corporation
233,Jul 18 2022 8:58AM,242044,16,"Emersa Waterbox, LLC"
234,Jul 18 2022 8:44AM,242037,10,ISDIN Corporation


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Jul 18 2022 9:14AM,242046,10,ISDIN Corporation,,59
1,Jul 18 2022 9:11AM,242050,20,Alumier Labs Inc.,,1
2,Jul 18 2022 9:10AM,242048,12,Hush Hush,,45
3,Jul 18 2022 9:09AM,242049,19,VITABIOTICS USA INC.,,1
4,Jul 18 2022 9:09AM,242047,12,Hush Hush,,2
5,Jul 18 2022 9:05AM,242045,10,ISDIN Corporation,,53
6,Jul 18 2022 9:05AM,242043,10,ISDIN Corporation,,61
7,Jul 18 2022 9:05AM,242041,10,ISDIN Corporation,,11
8,Jul 18 2022 8:58AM,242044,16,"Emersa Waterbox, LLC",,1
9,Jul 18 2022 8:44AM,242037,10,ISDIN Corporation,,53


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Jul 18 2022 9:14AM,242046,10,ISDIN Corporation,59,
1,Jul 18 2022 9:11AM,242050,20,Alumier Labs Inc.,1,
2,Jul 18 2022 9:10AM,242048,12,Hush Hush,45,
3,Jul 18 2022 9:09AM,242049,19,VITABIOTICS USA INC.,1,
4,Jul 18 2022 9:09AM,242047,12,Hush Hush,2,
5,Jul 18 2022 9:05AM,242045,10,ISDIN Corporation,53,
6,Jul 18 2022 9:05AM,242043,10,ISDIN Corporation,61,
7,Jul 18 2022 9:05AM,242041,10,ISDIN Corporation,11,
8,Jul 18 2022 8:58AM,242044,16,"Emersa Waterbox, LLC",1,
9,Jul 18 2022 8:44AM,242037,10,ISDIN Corporation,53,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Jul 18 2022 9:14AM,242046,10,ISDIN Corporation,59,
1,Jul 18 2022 9:11AM,242050,20,Alumier Labs Inc.,1,
2,Jul 18 2022 9:10AM,242048,12,Hush Hush,45,
3,Jul 18 2022 9:09AM,242049,19,VITABIOTICS USA INC.,1,
4,Jul 18 2022 9:09AM,242047,12,Hush Hush,2,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Jul 18 2022 9:14AM,242046,10,ISDIN Corporation,59,NaN
1,Jul 18 2022 9:11AM,242050,20,Alumier Labs Inc.,1,NaN
2,Jul 18 2022 9:10AM,242048,12,Hush Hush,45,NaN
3,Jul 18 2022 9:09AM,242049,19,VITABIOTICS USA INC.,1,NaN
4,Jul 18 2022 9:09AM,242047,12,Hush Hush,2,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Jul 18 2022 9:14AM,242046,10,ISDIN Corporation,59,0.0
1,Jul 18 2022 9:11AM,242050,20,Alumier Labs Inc.,1,0.0
2,Jul 18 2022 9:10AM,242048,12,Hush Hush,45,0.0
3,Jul 18 2022 9:09AM,242049,19,VITABIOTICS USA INC.,1,0.0
4,Jul 18 2022 9:09AM,242047,12,Hush Hush,2,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,3872399,358,16.0
12,484095,47,0.0
16,242044,1,0.0
19,726096,3,0.0
20,242050,1,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,3872399,358,16.0
1,12,484095,47,0.0
2,16,242044,1,0.0
3,19,726096,3,0.0
4,20,242050,1,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,358,16.0
1,12,47,0.0
2,16,1,0.0
3,19,3,0.0
4,20,1,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,358.0
1,12,Released,47.0
2,16,Released,1.0
3,19,Released,3.0
4,20,Released,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,16,19,20
Status,,,,,
Executing,16.0,0.0,0.0,0.0,0.0
Released,358.0,47.0,1.0,3.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,16,19,20
0,Executing,16.0,0.0,0.0,0.0,0.0
1,Released,358.0,47.0,1.0,3.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,16,19,20
0,Executing,16.0,0.0,0.0,0.0,0.0
1,Released,358.0,47.0,1.0,3.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()